# \[Week5] 추천시스템  
## [캐글 데이터](https://www.kaggle.com/tmdb/tmdb-movie-metadata?select=tmdb_5000_movies.csv) tmdb_5000_movies.csv   
참고 사이트 : [[구름 Edu]](https://edu.goorm.io/learn/lecture/20671/%EB%94%A5%EB%9F%AC%EB%8B%9D-%EC%9D%B4%EB%A1%A0-%EB%B0%8F-%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%8B%A4%EC%8A%B5)

## 1. 자연어처리 - 문서 유사도 (용어 정리)

1. Bag of Words   

\[답변] 단어들의 순서는 전혀 고려하지 않고, 단여들의 출현 빈도(frequency)에만 집중하는 텍스트 데이터의 수치화 표현 방법.  
ex) not bad not good -> not 2, bad 1, good 1

1) 문장 간의 유사도 구하는 방법 

\[답변] 각각의 인덱스에 있는 값을 전부 다 곱해준 다음에 다 더한다.  
awsome thank you [1,1,1,0,0,0,0]  
great thank you [0,1,1,1,0,0,0]  
=> 1x0 + 1x1 + 1x1 + 0x1 + 0x0 + 0x0 + 0x0 = 2

2) 단점

\[답변] 
1. **Sparsity**  
실제 문장 하나를 표현할 때 무수히 많은 0, 1의 빈도는 아주 작음 -> 많은 계산과 저장 공간이 필요함
2. **Frequennt words has more power**  
the man like the girl & the man love the girl의 유사도  
the man like the girl & the the the the the의 유사도
3. **Ignoring word orders**  
home run VS run home -> 같은 벡터로 표시 => 문맥이 무시 됨
4. **Out of vocabulary**  
실제 머신러닝을 돌릴 때, 오타와 줄임말들을 데이터로 갖고 있기 힘들끼 때문에 bag of words를 사용할 경우 처리가 난감해짐

2. n-그램

\[답변] 연속적인 N개의 토큰(단어, character)으로 구성된 것

**(예제) fine thank you**  
1) 1-gram

    1-1) Word level  

    [답변]  [fine, thank, you]

    1-2) Character level

    [답변]  [f,i,n,e, ,t,h,a,n,k, ,y,o,u]

2) 2-gram

    2-1) Word level

    [답변]  [fine thank, thank you]

    2-2) Character level 

    [답변] [fi,in,ne,e , t,th,ha,an,nk,k , y,yo,ou]

3) 3-gram

    3-1) Word level 

    [답변] [fine thank you]

    3-2) Character level 

    [답변] [fin,ine,ne ,e t, th,tha,han,ank,nk ,k y, yo,you]

4) n-gram 사용하는 이유?  

\[답변]
* Overcome bag of words drawbacks _(ignoring sequence of words)_
* Next word prediction
* Find Misspelling
* And more ...

3. 점 A(3,4), 점 B(4,3)의 cosine 유사도를 구하시오.  

\[답변] (3*4)+(4*3)/5*5 = cos(16.26도)

## 2. 추천시스템  
### 아래 코드에서 pass 부분을 지우고 그 자리에 코드 작성해주세요

### 0. 추천시스템 종류 정리

1. Content-based filtering  

\[답변] 사용자가 특정 아이템을 선호하는 경우 그 아이템과 비슷한 콘텐츠를 가진 다른 아이템을 추천해주는 방식. 예를 들어, A가 item a에 굉장히 높은 평점을 주었는데 a가 '이수진'이라는 감독이었으면 '이수진'감독의 다른 영화를 추천해주는 것.

2. Collaborative filtering  

\[답변] 많은 사용자들로부터 얻은 기호정보에 따라 사용자들의 관심사를 자동적으로 예측하게 해주는 방법. 예를 들어, 음악에 관한 추천시스템은 사용자들의 기호에 대한 부분적인 list를 이용하여 그 사용자의 음악에 대한 기호를 예측. 비슷한 취향을 가진 고객들에게 서로 아직 구매하지 않은 상품들은 교차 추천하거난 분류된 고객의 생활 형태나 취향에 따라 관 상품을 추천

    - user-based collaborative filtering  

    [답변] cosine similarity등을 이용하여 추천을 해주고자 하는 user(A)와 가장 비슷한 user(B)를 찾고 B는 봤으나 A는 보지 않은 영화를 추천해주는 방식

    - item-based collaborative filtering  

    [답변] User 4번이 개츠비 영화를 봤을 때, 우리가 개츠비와 타이타닉이 비슷하다는 것을 안다고 가정하고 타이타닉을 추천해주는 방식. 각각의 영화가 유저라는 컬럼의 벡터로 표현되어서 영화끼리의 유사도를 구할 수 있음.

Content-based filtering을 해봅시다.  
보통은 장르를 기준으로 많이 하지만, 자연어처리에 좀 더 익숙해지기 위해서 줄거리를 바탕으로 해보겠습니다.  
Collaborative filtering도 한번 직접 해보세요. 재밌습니다:)

### 1. CSV 파일 불러와 DataFrame으로 저장

In [1]:
import pandas as pd
movies = pd.read_csv('./tmdb_5000_movies.csv')          # tmdb_5000_movies.csv dataframe으로 읽어오기
movies = movies[["original_title", "overview"]]
movies["overview"] = movies["overview"].astype("str")
movies

,original_title,overview
0,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,Spectre,A cryptic message from Bond’s past sends him o...
3,The Dark Knight Rises,Following the death of District Attorney Harve...
4,John Carter,"John Carter is a war-weary, former military ca..."
...,...,...
4798,El Mariachi,El Mariachi just wants to play his guitar and ...
4799,Newlyweds,A newlywed couple's honeymoon is upended by th...
4800,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic..."
4801,Shanghai Calling,When ambitious New York attorney Sam is sent t...


### 2. 전처리  
출력 예시)

    original_title    0
    overview          3
    dtype: int64


In [2]:
# null 값 개수 확인
movies.isnull().sum()

original_title    0
overview          0
dtype: int64

1) 줄거리가 NaN인 **영화** drop

In [3]:
movies.overview.dropna()

0       In the 22nd century, a paraplegic Marine is di...
1       Captain Barbossa, long believed to be dead, ha...
2       A cryptic message from Bond’s past sends him o...
3       Following the death of District Attorney Harve...
4       John Carter is a war-weary, former military ca...
                              ...                        
4798    El Mariachi just wants to play his guitar and ...
4799    A newlywed couple's honeymoon is upended by th...
4800    "Signed, Sealed, Delivered" introduces a dedic...
4801    When ambitious New York attorney Sam is sent t...
4802    Ever since the second grade when he first saw ...
Name: overview, Length: 4803, dtype: object

2) "overview" column 모두 소문자로, 문자+숫자(\w)만 남기고 나머지는 띄어쓰기로 대체

In [4]:
import re
# 되도록 apply, lambda 사용할것!
movies['overview'] = movies['overview'].astype(str).apply(lambda x:str.lower(x.replace('[\w]','')))

### 3. TF-IDF 계산

1) TF 란?  

\[답변] how frequently a term occurs in a document  
어떤 문서에서 등장한 특정 단어의 횟수

2) DF 란?  

\[답변] 특정 단어가 등장한 문서의 수

3) IDF 란?   

\[답변] log(total # of docs / # of Docs with the term in it)  
df에 반비례하는 수 

라이브러리로 적용해보자...

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
# ngram_range=(1, 2) 는 단어를 1개 혹은 2개 연속으로 보겠다
tfidf_vec =TfidfVectorizer()                    # Vectorizer 생성
tfidf_matrix = tfidf_vec.fit_transform(movies['overview'])      # Vectorizer가 단어들을 학습

### 4. 영화 간 **cosine 유사도**  
\[출력예시]  

    ### COSINE Similarity ###
    [[1.         0.01513636 0.00614044 ... 0.01195144 0.00571986 0.00630023]
    [0.01513636 1.         0.01307772 ... 0.01768395 0.00997186 0.00666392]
    [0.00614044 0.01307772 1.         ... 0.01288316 0.00565125 0.00612543]
    ...
    [0.01195144 0.01768395 0.01288316 ... 1.         0.01532266 0.0089986 ]
    [0.00571986 0.00997186 0.00565125 ... 0.01532266 1.         0.01649368]
    [0.00630023 0.00666392 0.00612543 ... 0.0089986  0.01649368 1.        ]]
    [[   0 3603  634 ... 2852 3945 3386]
    ### 유사도 기준 index 정렬 ###
    [[   0 3603  634 ... 2852 3945 3386]
    [   1 2379 2542 ... 3487 3622 3945]
    [   2 1343 3161 ... 3534 3637 3082]
    ...
    [4797 4033  569 ... 2604 3685 1658]
    [4798 2017 1480 ... 2108 1658 4511]
    [4799 2586  868 ... 3987 2192  669]]



In [6]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
plot_similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)         # 줄거리 간 cosine 유사도 구하기 - tfidf_matrix 사용
print("### COSINE Similarity ###")
print(plot_similarity)

similar_index = np.argsort(-plot_similarity) # 유사도 높은 순서대로 index 정렬
print("### 유사도 기준 index 정렬 ###") 
print(similar_index)


### COSINE Similarity ###
[[1.         0.03668199 0.01946126 ... 0.0327237  0.01739194 0.01366935]
 [0.03668199 1.         0.04345693 ... 0.0486506  0.03179655 0.02105829]
 [0.01946126 0.04345693 1.         ... 0.04058832 0.01700339 0.0182645 ]
 ...
 [0.0327237  0.0486506  0.04058832 ... 1.         0.04631238 0.02386989]
 [0.01739194 0.03179655 0.01700339 ... 0.04631238 1.         0.04716964]
 [0.01366935 0.02105829 0.0182645  ... 0.02386989 0.04716964 1.        ]]
### 유사도 기준 index 정렬 ###
[[   0 3604  529 ... 3152 1038 4401]
 [   1 2542 3095 ... 3276 4401 1658]
 [   2 1343 4071 ... 1011 3493 4548]
 ...
 [4800 4034  633 ... 2656 3686 4274]
 [4801 2027 1796 ... 4431 2656 4140]
 [4802  868 4100 ... 4075 1273 3988]]


In [7]:
input_movie = "Avatar"          # data에 있는 영화의 제목을 넣어야 합니다.

movie_index = movies[movies.original_title == input_movie].index[0] # input_movie에 해당하는 index 값 가져오기

similar_movies = similar_index[movie_index][1:11]  # 유사도 상위 10개 index 가져오기

# 인덱스로 사용하기 위해서는 1차원으로 변형
similar_movies_index = [i for i in similar_movies]
# similar_movies 1차원 변형
display(movies.iloc[similar_movies_index])

,original_title,overview
3604,Apollo 18,"officially, apollo 17 was the last manned miss..."
529,Tears of the Sun,navy seal lieutenant a.k. waters and his elite...
2130,The American,dispatched to a small italian town to await fu...
1341,Obitaemyy Ostrov,"on the threshold of 22nd century, furrowing th..."
634,The Matrix,"set in the 22nd century, the matrix tells the ..."
2628,Blood and Chocolate,a young teenage werewolf is torn between honor...
847,Semi-Pro,"jackie moon is the owner, promoter, coach, and..."
311,The Adventures of Pluto Nash,"the year is 2087, the setting is the moon. plu..."
942,The Book of Life,"the journey of manolo, a young man who is torn..."
3458,Duel in the Sun,beautiful half-breed pearl chavez becomes the ...


3주동안의 자연어처리 스터디가 끝났습니다. 다음주부터는 이미지처리 스터디가 시작됩니다.  
짧은 기간이라 많은 부분을 다루지는 못했지만, 추후에 프로젝트를 진행하면서 멤버들끼리 딥러닝 등 심화 부분을 공부하시면 더 좋을 것 같습니다.  
중간에 시험기간도 겹쳐서 바쁜 와중에도 과제 잘 해주셔서 감사합니다!